In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

25/08/29 13:08:14 WARN Utils: Your hostname, EmmanuelPC resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/08/29 13:08:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/29 13:08:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/29 13:08:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/08/29 13:08:16 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [17]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark.readStream \
    .format('socket') \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

25/08/29 13:26:15 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [13]:
# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)


In [14]:
# Generate running word count
wordCounts = words.groupBy("word").count()

In [16]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

25/08/29 13:21:16 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-22a4358f-3254-467e-b3a8-6c3e3416faaf. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/08/29 13:21:16 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/08/29 13:21:17 ERROR MicroBatchExecution: Query [id = 8f3bff47-0418-4e52-b6a4-6c409e64949f, runId = f9c0fa26-5ffe-46cf-994f-709164d252c8] terminated with error
java.net.ConnectException: Connection refused (Connection refused)
	at java.base/java.net.PlainSocketImpl.socketConnect(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.doConnect(AbstractPlainSocketImpl.java:412)
	at java.base/java.net.AbstractPlainSocketImpl.connectToAddress(AbstractPlainSocke

StreamingQueryException: [STREAM_FAILED] Query [id = 8f3bff47-0418-4e52-b6a4-6c409e64949f, runId = f9c0fa26-5ffe-46cf-994f-709164d252c8] terminated with exception: Connection refused (Connection refused)